In [19]:
## Installing Libraries
!pip install langchain langchain-core langchain-community langchain-huggingface requests -q


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
## Imorting necessary libraries
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.agents import initialize_agent, Tool, tool
from langchain.agents.agent_types import AgentType
from langchain_core.messages import HumanMessage
import requests
import json
import functools

In [38]:
## Creating a python tool 

@tool
def multiply(a: int, b: int) -> int:
    '''Given two numbers a and b this tool returns their product'''
    return a * b

In [39]:
## Tool testing
print(multiply.invoke({'a' : 3, 'b' : 4}))
print(multiply.name)
print(multiply.description)
print(multiply.args)

12
multiply
Given two numbers a and b this tool returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [40]:
## Initializing HuggingFace LLM
llm = HuggingFaceEndpoint(
    repo_id= 'meta-llama/Llama-3.3-70B-Instruct'
)

## Initializing model
model = ChatHuggingFace(llm = llm)

In [41]:
## tool binding with LLM
llm_with_tools = model.bind_tools([multiply])

In [42]:
## Defining a human message
query = HumanMessage('Can you multiply 3 with 128 ?')

In [43]:
## Creating a list that stores HumanMessage, AIMessage and ToolMessage that can be used for historical information and chatbot generation
message = [query]
message

[HumanMessage(content='Can you multiply 3 with 128 ?', additional_kwargs={}, response_metadata={})]

In [44]:
## Tool calling
response = llm_with_tools.invoke(message)
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": "3", "b": "128"}', 'name': 'multiply', 'description': None}, 'id': 'chatcmpl-tool-ad3021a52c334b9ea61450b2726cb770', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 204, 'total_tokens': 226}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4add37cb-f3f6-433f-aff2-6344ef589868-0', tool_calls=[{'name': 'multiply', 'args': {'a': '3', 'b': '128'}, 'id': 'chatcmpl-tool-ad3021a52c334b9ea61450b2726cb770', 'type': 'tool_call'}], usage_metadata={'input_tokens': 204, 'output_tokens': 22, 'total_tokens': 226})

In [45]:
## Appending the AIMessage into the aobve defined list
message.append(response)
message

[HumanMessage(content='Can you multiply 3 with 128 ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": "3", "b": "128"}', 'name': 'multiply', 'description': None}, 'id': 'chatcmpl-tool-ad3021a52c334b9ea61450b2726cb770', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 204, 'total_tokens': 226}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4add37cb-f3f6-433f-aff2-6344ef589868-0', tool_calls=[{'name': 'multiply', 'args': {'a': '3', 'b': '128'}, 'id': 'chatcmpl-tool-ad3021a52c334b9ea61450b2726cb770', 'type': 'tool_call'}], usage_metadata={'input_tokens': 204, 'output_tokens': 22, 'total_tokens': 226})]

In [46]:
## Since the reposnse structure we are getting is incompatable to what langchain's invoke() method accepts, we are reconstructing a function by defining a new function:
def extract_and_normalize_tool_call(response):
    """Extract the first tool call from LLM output and normalize it for LangChain invoke"""
    tool_call_raw = response.additional_kwargs.get("tool_calls", [])[0]

    return {
        "name": tool_call_raw["function"]["name"],
        "args": json.loads(tool_call_raw["function"]["arguments"]),
        "description": tool_call_raw["function"].get("description"),
        "id": tool_call_raw["id"],
        "type": "tool_call"
    }

In [47]:
## Getting the required structured output from LLM
structured_tool_call = extract_and_normalize_tool_call(response=response)
structured_tool_call

{'name': 'multiply',
 'args': {'a': '3', 'b': '128'},
 'description': None,
 'id': 'chatcmpl-tool-ad3021a52c334b9ea61450b2726cb770',
 'type': 'tool_call'}

In [48]:
## Sending to LLM and getting the output
tool_result = multiply.invoke(structured_tool_call)
tool_result

ToolMessage(content='384', name='multiply', tool_call_id='chatcmpl-tool-ad3021a52c334b9ea61450b2726cb770')

In [49]:
## Appending the ToolMessage into the aobve defined list
message.append(tool_result)
message

[HumanMessage(content='Can you multiply 3 with 128 ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": "3", "b": "128"}', 'name': 'multiply', 'description': None}, 'id': 'chatcmpl-tool-ad3021a52c334b9ea61450b2726cb770', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 204, 'total_tokens': 226}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4add37cb-f3f6-433f-aff2-6344ef589868-0', tool_calls=[{'name': 'multiply', 'args': {'a': '3', 'b': '128'}, 'id': 'chatcmpl-tool-ad3021a52c334b9ea61450b2726cb770', 'type': 'tool_call'}], usage_metadata={'input_tokens': 204, 'output_tokens': 22, 'total_tokens': 226}),
 ToolMessage(content='384', name='multiply', tool_call_id='chatcmpl-tool-ad3021a52c334b9ea61450b2726cb770')]

In [50]:
## We have maintained the entire chat history that helps us to call and evaluate the calculation using the tool, now we just need to pass the entire message thread to the LLM and get the response back.
final_result = llm_with_tools.invoke(message)
print(final_result.content)

This JSON function call will multiply 3 and 128, returning their product, which is 384.
